- Selenium만으로 유튜브랭킹 수집하기

In [1]:
import requests
import pandas as pd

In [3]:
!apt-get update > /dev/null
!pip install selenium > /dev/null
!apt install chromium-chromedriver > /dev/null

In [4]:
import time
from selenium import webdriver
from selenium.webdriver.common.keys import Keys

In [58]:
options = webdriver.ChromeOptions()
options.add_argument('--headless')   # 화면없이 실행
options.add_argument('--no-sandbox')
options.add_argument("--single-process")
options.add_argument("--disable-dev-shm-usage")
driver = webdriver.Chrome('chromedriver', options=options)

In [59]:
def convert_unit(s):
  #s = ''.join(s.split('억'))
  s = s.replace('억', '').replace('개','').replace(',', '')
  s = s.replace('만', '0000')
  return f'{int(s):,d}'
    

In [60]:
trs = driver.find_elements_by_css_selector('.aos-init')
len(trs)

0

In [61]:
from tqdm import notebook # 진척도 보는 함수

In [62]:

results = []

for page in notebook.tqdm(range(1,11)):
  url = 'https://youtube-rank.com/board/bbs/board.php?bo_table=youtube&page='+str(page)
  driver.get(url)
  time.sleep(2)
  trs = driver.find_elements_by_css_selector('.aos-init')
  body = driver.find_element_by_css_selector('body')


  for tr in trs:
    body.send_keys(Keys.PAGE_DOWN)  # 화면을 내려야 데이터가 읽어짐
    category = tr.find_element_by_css_selector('p.category').text.strip(' \n[]')
    name = tr.find_element_by_css_selector('.subject a').text.strip(' \n[]')
    subscriber = convert_unit(tr.find_element_by_css_selector('.subscriber_cnt').text)
    view = convert_unit(tr.find_element_by_css_selector('.view_cnt').text)
    video = convert_unit(tr.find_element_by_css_selector('.video_cnt').text)

    results.append([category,name,subscriber,view,video])


  0%|          | 0/10 [00:00<?, ?it/s]

In [63]:
df = pd.DataFrame(results, columns=['카테고리', '채널명', '구독자수', '뷰수', '비디오수'])
df

,카테고리,채널명,구독자수,뷰수,비디오수
0,음악/댄스/가수,BLACKPINK,"64,000,000","19,003,810,000",371
1,음악/댄스/가수,HYBE LABELS,"60,300,000","18,713,050,000",654
2,음악/댄스/가수,BANGTANTV,"56,400,000","12,194,960,000","1,579"
3,음악/댄스/가수,SMTOWN,"28,500,000","21,870,920,000","3,729"
4,키즈/어린이,Boram Tube Vlog [보람튜브 브이로그,"26,500,000","11,052,880,000",223
...,...,...,...,...,...
995,게임,미소,"510,000","210,770,000","3,084"
996,미분류,밉지않은 관종언니,"510,000","67,740,000",170
997,미분류,복지마블TV [Welfare Marble,"510,000","49,960,000",240
998,음식/요리/레시피,Muggo,"510,000","183,720,000","1,388"


In [64]:
df.to_csv('유튜브랭크1000_셀레늄.csv', index=None) # index=False 도 됨

In [65]:
driver.close()